## Clase TSTR


In [ ]:
from typing import *

### Métodos  proxy definidos a *Mano*

In [ ]:
# Este código base es del libro fuzzing book
class tstr(str):
    """Wrapper for strings, saving taint information"""

    def __new__(cls, value, *args, **kw):
        """Create a tstr() instance. Used internally."""
        return str.__new__(cls, value)

    def __init__(self, value: Any, taint: Any = None, **kwargs) -> None:
        """Constructor.
        `value` is the string value the `tstr` object is to be constructed from.
        `taint` is an (optional) taint to be propagated to derived strings."""
        self.taint: Any = taint

    def __repr__(self):
        """Return a representation."""
        return tstr(str.__repr__(self), taint=self.taint)

    def __str__(self) -> str:
        """Convert to string"""
        return str.__str__(self)

    def clear_taint(self):
        """Remove taint"""
        self.taint = None
        self.trace = []
        return self

    def has_taint(self):
        """Check if taint is present"""
        return self.taint is not None

    def create(self, s):
        return tstr(s, taint=self.taint)

    def __radd__(self, value):
        """Return value + self, as a `tstr` object"""
        return self.create(value + str(self))

    @staticmethod
    def make_str_wrapper(fun, name):
        """Make `fun` (a `str` method) a method in `tstr`"""
        def proxy(self, *args, **kwargs):
            res = fun(self, *args, **kwargs)
            return self.create(res)

        if hasattr(fun, '__doc__'):
            # Copy docstring
            proxy.__doc__ = fun.__doc__

        return proxy

### Métodos Proxy Autogenerados

In [ ]:
def informationflow_init_1():
    for name in ['__format__', '__mod__', '__rmod__', '__getitem__',
                 '__add__', '__mul__', '__rmul__',
                 'capitalize', 'casefold', 'center', 'encode',
                 'expandtabs', 'format', 'format_map', 'join',
                 'ljust', 'lower', 'lstrip', 'replace',
                 'rjust', 'rstrip', 'strip', 'swapcase', 'title', 'translate', 'upper']:
        fun = getattr(str, name)
        setattr(tstr, name, tstr.make_str_wrapper(fun,name))
informationflow_init_1()
INITIALIZER_LIST = [informationflow_init_1]
def initialize():
    for fn in INITIALIZER_LIST:
        fn()

initialize()

### Tests

In [ ]:
# Code: test1
def doo(a):
  # line 1
  return a +"a" # line 2


input = tstr("hello", taint ='LOW')
doo(input)
input.trace

AttributeError: 'tstr' object has no attribute 'trace'

In [ ]:
# Code: test2
def imprimir(a):
  print("value: " + a) # line 1

def bar(a):
  c = a[:1]  # line 1
  imprimir(c)     # line 2
  # line 3
  imprimir(a + c) # line 4

def foo(a):
  # line 1
  return bar(a) # line 2


input2 = tstr("hello,")
foo(input2)
input2.trace

value: h
value: hello,h


AttributeError: 'tstr' object has no attribute 'trace'

In [ ]:
# Code: test3
from urllib.parse import urlparse

url_string = tstr("http://www.google.com/search?q=fuzzing")
urlparse(url_string)

url_string.trace

AttributeError: 'tstr' object has no attribute 'trace'

In [ ]:
import re


email_pattern = tstr(r'\S+@\S+\.\S+')
text = "Contact us at support@example.com or info@example.org"

re.findall(email_pattern, text)

email_pattern.trace

AttributeError: 'tstr' object has no attribute 'trace'

## Testing the html report

In [ ]:
from collections import defaultdict
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import *
import inspect

def to_html(trace):
  # complete la función, puede crear mas funciones auxiliares si necesita
  pass

In [ ]:
from IPython.display import HTML, display

def imprimir(a):
  print("value: " + a) # line 1

def bar(a):
  c = a[:1]  # line 1
  imprimir(c)     # line 2
  # line 3
  imprimir(a + c) # line 4

def foo(a):
  # line 1
  return bar(a) # line 2


html_input = tstr("hello sdfgsdf")
foo(html_input)


display(HTML(to_html(html_input.trace)))

value: h
value: hello sdfgsdfh


AttributeError: 'tstr' object has no attribute 'trace'